# **Program Untuk Pengambilan Data Video**

Program berbasis jupyter ini dibuat untuk menjalankan proses ekstraksi fitur data skeletal numerik dari Mediapipe yang diperoleh dari dataset video.

---

## Proses _import library_

In [66]:
import cv2 as opencv
import numpy
import pandas
import os
from pathlib import Path
import mediapipe
from mediapipe.framework.formats import landmark_pb2

## Pengaturan Mediapipe Holistic

In [67]:
mediapipe_holistic = mediapipe.solutions.holistic
holistic = mediapipe_holistic.Holistic(
    static_image_mode=False,
    model_complexity=1,
    smooth_landmarks=True,
    enable_segmentation=False,
    refine_face_landmarks=False,
    min_detection_confidence=0.25,
    min_tracking_confidence=0.25,
)

# Initialize mediapipe detection settings
mediapipe_holistic = mediapipe.solutions.holistic
mediapipe_drawing = mediapipe.solutions.drawing_utils
mediapipe_drawing_styles = mediapipe.solutions.drawing_styles

## Inisialisasi _function_

### _Function_ penghitung jarak _euclidean_

In [68]:
def euclidean_distance(pointA, pointB):
    return ((pointA.x - pointB.x) ** 2 + (pointA.y - pointB.y) ** 2) ** 0.5

### _Function_ normalisasi landmark

In [69]:
def normalize_landmarks(landmarks, shoulder_center_point, shoulder_width):
    if not landmarks or shoulder_width == 0:
        return [(0, 0)] * len(landmarks)
    return [
        ((landmark_point.x - shoulder_center_point[0]) / shoulder_width,
         (landmark_point.y - shoulder_center_point[1]) / shoulder_width
        ) for landmark_point in landmarks
    ]

### _Function_ ekstraksi fitur _landmark_ dari frame melalui Mediapipe

In [70]:
def extract_landmarks_from_frame(frame, video_path):
    frame_rgb = opencv.cvtColor(frame, opencv.COLOR_BGR2RGB)
    frame_rgb.flags.writeable = False
    detection_results = holistic.process(frame_rgb)
    frame_rgb.flags.writeable = True

    output_frame = opencv.cvtColor(frame_rgb, opencv.COLOR_RGB2BGR)

    if not detection_results.pose_landmarks:
        return None

    pose_landmark_1_to_16 = detection_results.pose_landmarks.landmark[:17]
    left_shoulder = detection_results.pose_landmarks.landmark[mediapipe_holistic.PoseLandmark.LEFT_SHOULDER]
    right_shoulder = detection_results.pose_landmarks.landmark[mediapipe_holistic.PoseLandmark.RIGHT_SHOULDER]
    
    shoulder_center_point = (
        (left_shoulder.x + right_shoulder.x) / 2,
        (left_shoulder.y + right_shoulder.y) / 2
    )
    
    shoulder_width = euclidean_distance(left_shoulder, right_shoulder)

    normalized_custom_pose = normalize_landmarks(
        pose_landmark_1_to_16, 
        shoulder_center_point, 
        shoulder_width
    )

    normalized_right_hand = normalize_landmarks(
        detection_results.right_hand_landmarks.landmark, 
        shoulder_center_point, 
        shoulder_width
    ) if detection_results.right_hand_landmarks else [(0, 0)] * 21

    normalized_left_hand = normalize_landmarks(
        detection_results.left_hand_landmarks.landmark,
        shoulder_center_point,
        shoulder_width
    ) if detection_results.left_hand_landmarks else [(0, 0)] * 21

    extracted_frame_landmarks = []
    for landmark_x, landmark_y in normalized_custom_pose + normalized_right_hand + normalized_left_hand:
        extracted_frame_landmarks.extend([landmark_x, landmark_y])

    filtered_pose = landmark_pb2.NormalizedLandmarkList(
        landmark = [detection_results.pose_landmarks.landmark[i] for i in range(17)]
    )

    filtered_connections = [
        connection for connection in mediapipe_holistic.POSE_CONNECTIONS if connection[0] < 17 and connection[1] < 17
    ]

    # === Draw landmarks directly on the frame to return ===
    mediapipe_drawing.draw_landmarks(
        output_frame,
        filtered_pose,
        filtered_connections,
        landmark_drawing_spec=mediapipe_drawing_styles.get_default_pose_landmarks_style()
    )
    if detection_results.right_hand_landmarks:
        mediapipe_drawing.draw_landmarks(
            output_frame,
            detection_results.right_hand_landmarks,
            mediapipe_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mediapipe_drawing_styles.get_default_hand_landmarks_style()
        )
    if detection_results.left_hand_landmarks:
        mediapipe_drawing.draw_landmarks(
            output_frame,
            detection_results.left_hand_landmarks,
            mediapipe_holistic.HAND_CONNECTIONS,
            landmark_drawing_spec=mediapipe_drawing_styles.get_default_hand_landmarks_style()
        )
    
    opencv.putText(
        output_frame,
        f'Name : {video_path}',
        (10, 30),
        opencv.FONT_HERSHEY_SIMPLEX,
        0.75,
        (255,255,255),
        2,
        opencv.LINE_AA,
    )

    return extracted_frame_landmarks, output_frame

## Proses Utama

### Menentukan lokasi _folder input_ dan _folder output_

In [80]:
input_folder = Path("input_videos/number/")
output_folder = Path("output_csv")
output_folder.mkdir(exist_ok=True)

In [81]:
MAX_FRAMES = None  # No max limit for variable-length
SEQUENCE_COUNT = 0

In [82]:
for label_folder in input_folder.iterdir():
    if not label_folder.is_dir():
        continue
    label = label_folder.name

    for video_path in label_folder.glob("*.mp4"):
        video_file = opencv.VideoCapture(str(video_path))
        sequence = []

        while True:
            available, frame = video_file.read()

            if not available:
                break

            frame_data, output_frame = extract_landmarks_from_frame(frame, video_path)
            if frame_data:
                sequence.append(frame_data)

                opencv.imshow("Ekstrasi Fitur Data Skeletal Mediapipe Holistic", output_frame)

                if opencv.waitKey(10) & 0xFF == ord('q'):
                    break
            
        video_file.release()
        opencv.destroyAllWindows()

        if not sequence:
            print(f"[INFO] Skipped (no pose): {video_path.name}")
            continue

        data_frame = pandas.DataFrame(sequence)
        output_file = output_folder / f"{label}_{SEQUENCE_COUNT:03}.csv"
        data_frame.to_csv(output_file, index=False, header=False)
        SEQUENCE_COUNT += 1
        print(f"[SUCCESS] Saved variable-length sequence: {output_file}")

print(f"\n[INFO] Total sequences saved: {SEQUENCE_COUNT}")

[SUCCESS] Saved variable-length sequence: output_csv\1_000.csv
[SUCCESS] Saved variable-length sequence: output_csv\10_001.csv
[SUCCESS] Saved variable-length sequence: output_csv\11_002.csv
[SUCCESS] Saved variable-length sequence: output_csv\12_003.csv
[SUCCESS] Saved variable-length sequence: output_csv\13_004.csv
[SUCCESS] Saved variable-length sequence: output_csv\14_005.csv
[SUCCESS] Saved variable-length sequence: output_csv\15_006.csv
[SUCCESS] Saved variable-length sequence: output_csv\16_007.csv
[SUCCESS] Saved variable-length sequence: output_csv\17_008.csv
[SUCCESS] Saved variable-length sequence: output_csv\18_009.csv
[SUCCESS] Saved variable-length sequence: output_csv\19_010.csv
[SUCCESS] Saved variable-length sequence: output_csv\2_011.csv
[SUCCESS] Saved variable-length sequence: output_csv\20_012.csv
[SUCCESS] Saved variable-length sequence: output_csv\3_013.csv
[SUCCESS] Saved variable-length sequence: output_csv\30_014.csv
[SUCCESS] Saved variable-length sequence: o